# Model Evaluation for L2RPN

In [45]:
import grid2op
from grid2op.Agent.doNothing import DoNothingAgent
from grid2op.Runner import Runner
from grid2op.Episode import EpisodeData
from grid2op.Reward import RedispReward, IncreasingFlatReward, AlarmReward, CombinedScaledReward
from grid2op.utils import EpisodeStatistics, ScoreL2RPN2020, ScoreICAPS2021, ScoreL2RPN2022
from pathlib import Path
from typing import List, Dict
import pandas as pd

from tqdm.notebook import tqdm

## Settings

In [39]:
env_name: str = "rte_case14_realistic"
evaluation_folder_path: str = "/home/l2rpn/data/evaluation"
validation_suffix: str = "_val10"
difficulty = "competition"
number_of_episodes = 100



### Environment Initialization

In [40]:
# Initialize Environment
env = grid2op.make(env_name + validation_suffix, reward_class=CombinedScaledReward)

# Initialized Combined Reward similar to L2RPN Score
combined_reward: CombinedScaledReward = env.get_reward_instance() 
combined_reward.addReward("IncreasingFlat", IncreasingFlatReward(), 0.7)
combined_reward.addReward("Redispatching", RedispReward(), 0.7)
if env.alarms_area_lines:
    combined_reward.addReward("Alarm", AlarmReward(), 0.3)
combined_reward.initialize(env)

# Initialize Runner Parameters
runner_parameters = env.get_params_for_runner()
runner_parameters["verbose"] = True


### Agents

#### Do nothing

In [42]:
agent = DoNothingAgent(env.action_space)

## Generate Do Nothing Data

In [5]:
do_nothing_evaluation_folder: Path = Path(evaluation_folder_path, "do_nothing_" + env_name)
    
if not do_nothing_evaluation_folder.exists():
    do_nothing_evaluation_folder.mkdir(parents=True, exist_ok=False)
    runner = Runner(**runner_parameters, agentClass=DoNothingAgent)
    runner.run(nb_episode=number_of_episodes, path_save=str(do_nothing_evaluation_folder), pbar=tqdm)

episode:   0%|          | 0/100 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

...

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

episode:   0%|          | 0/8064 [00:00<?, ?it/s]

## Retrieving Data Generated by the Runner on Validation Set

In [29]:
agent_to_episodes: Dict[str, List[int]] = (
{agent_folder_path.name: 
 [ EpisodeData.from_disk(agent_folder_path, episode_folder_path.name)
  for episode_folder_path in tqdm(list(filter(lambda x: x.is_dir() and str(x.name).isdigit(), 
                        Path(evaluation_folder_path, agent_folder_path).iterdir())))] 
 for agent_folder_path in tqdm(list(filter(lambda x: x.is_dir(), 
                          Path(evaluation_folder_path).iterdir())))})

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

## Score

In [ ]:
# Score Settings
score_l2rpn2022 = ScoreL2RPN2022(env, nb_scenario=number_of_episodes)        

In [44]:
# Agent Score
agent_score = score_l2rpn2022.get(agent)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1097, 3399, 1099, 377, 809, 805, 519, 807, 520, 516, 1099, 391, 1096, 4, 521, 526, 520, 515, 522, 524, 1102, 522, 513, 1098, 815, 2534, 816, 804, 807, 805, 232, 1094, 524, 375, 803, 520, 4, 1388, 381, 803, 379, 521, 516, 517, 1096, 1099, 1099, 807, 2390, 523, 381, 523, 521, 523, 517, 379, 514, 518, 231, 808, 1094, 516, 510, 803, 521, 505, 239, 809, 234, 526, 808, 1085, 807, 811, 1385, 805, 811, 815, 791, 1384, 1097, 526, 231, 382, 813, 812, 516, 520, 4, 526, 515, 1383, 811, 1529, 808, 521, 5

In [54]:
# Save Score to file
agent_score_df = pd.DataFrame(agent_score).transpose()
agent_score_df.columns = ["all_scores", "ts_survived", "total_ts"]


,all_scores,ts_survived,total_ts
0,0.0,1097.0,8064.0
1,0.0,3399.0,8064.0
2,0.0,1099.0,8064.0
3,0.0,377.0,8064.0
4,0.0,809.0,8064.0
...,...,...,...
95,0.0,521.0,8064.0
96,0.0,511.0,8064.0
97,0.0,1102.0,8064.0
98,0.0,522.0,8064.0


In [58]:
agent_score_df.to_csv(Path(do_nothing_evaluation_folder, "l2rpn_2022_score.csv"))